# Policy Gradients

In policy-based methods, instead of learning a value function that tells us what is the expected sum of rewards given a state and an action, we learn directly the policy function that maps state to action (select actions without using a value function).

## Advantages of Policy Gradient over Deep Q-Learning:
- __Convergence__: 
  
  Policy-based methods have better convergence properties. The problem with value-based methods is that they can have a big oscillation while training. This is because the choice of action may change dramatically for an arbitrarily small change in the estimated action values. On the other hand, with policy gradient, we just follow the gradient to find the best parameters. We see a smooth update of our policy at each step.


- __Policy gradients are more effective in high dimensional action spaces or when using continuous actions__:
  
  The problem with Deep Q-learning is that their predictions assign a score (maximum expected future reward) for each possible action, at each time step, given the current state. On the other hand, in policy-based methods, you just adjust the parameters directly: thanks to that you’ll start to understand what the maximum will be, rather than computing (estimating) the maximum directly at every step.


- __Policy gradients can learn stochastic policies__:
  
  One of these is that we don’t need to implement an exploration/exploitation trade off. A stochastic policy allows our agent to explore the state space without always taking the same action. This is because it outputs a probability distribution over actions. 
  
  We also get rid of the problem of perceptual aliasing. Perceptual aliasing is when we have two states that seem to be (or actually are) the same, but need different actions.

## Disadvantages
Naturally, Policy gradients have one big disadvantage. A lot of the time, they converge on a local maximum rather than on the global optimum.

Instead of Deep Q-Learning, which always tries to reach the maximum, policy gradients converge slower, step by step. They can take longer to train.

## Policy Search

We have to maximize score function $J(\theta) = \mathbb{E}_{\pi}(\sum \lambda r)$. The main idea here is that $J(\theta)$ will tell us how good our $\pi$ is. Policy gradient ascent will help us to find the best policy parameters to maximize the sample of good actions.

Our score can also be defined as:
$$J(\theta) = \mathbb{E}_{\pi} (R(\tau)) = \sum_{\tau} \pi(\tau \mid \theta) R(\tau)$$
where $\tau$ is route.

Then we can calculate gradient:
$$
\begin{align}
\nabla_{\theta} J(\theta) &= \nabla_{\theta} \sum_{\tau} \pi(\tau \mid \theta) R(\tau)\\
&= \sum_{\tau} \nabla_{\theta} \pi(\tau \mid \theta) R(\tau)\\
&= \sum_{\tau} \pi(\tau \mid \theta) \frac{\nabla_{\theta} \pi(\tau \mid \theta)}{\pi(\tau \mid \theta)} R(\tau)\\
&= \sum_{\tau} \pi(\tau \mid \theta) \nabla_{\theta} (\log \pi(\tau \mid \theta)) R(\tau)\\
&= \mathbb{E}_{\pi} [\nabla_{\theta} (\log \pi(\tau \mid \theta)) R(\tau)]
\end{align}
$$

So that our update rule is: 
$$\Delta \theta = \alpha \nabla_{\theta} (\log \pi(\tau \mid \theta)) R(\tau)$$

## Monte Carlo Policy Gradient

```
Initialize θ
for each episode τ = S0, A0, R1, S1, …, ST:
    for t <-- 1 to T-1:
        Δθ = α ∇theta(log π(St, At, θ)) Gt
        θ = θ + Δθ
```
But we face a problem with this algorithm. Because we only calculate R at the end of the episode, we average all actions. Even if some of the actions taken were very bad, if our score is quite high, we will average all the actions as good.

This can be fixed using Actor Critic (a hybrid between value-based algorithms and policy-based algorithms).

In [22]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import numpy as np

import gym

In [23]:
RANDOM_SEED = 40

np.random.seed(RANDOM_SEED)
tf.set_random_seed(RANDOM_SEED)

In [24]:
env = gym.make("CartPole-v0")

a_size = env.action_space.n
s_size = env.observation_space.shape[0]
print("Action space size: {}".format(a_size))
print("State space size: {}".format(s_size))

possible_actions = np.identity(a_size)

Action space size: 2
State space size: 4


In [25]:
def discount_rewards(episode_rewards, gamma=0.95):
    discounted_episode_rewards = np.zeros_like(episode_rewards)
    cumulative = 0
    for i in range(len(episode_rewards) - 1, -1, -1):
        cumulative = cumulative * gamma + episode_rewards[i]
        discounted_episode_rewards[i] = cumulative
    return discounted_episode_rewards

In [26]:
class PolicyNetwork(object):
    def __init__(self, s_size, a_size, learning_rate=0.01):
        self.s_size = s_size
        self.a_size = a_size
        
        self.states = tf.placeholder(shape=[None, self.s_size], dtype=tf.float32)
        self.dense = tf.layers.dense(inputs=self.states, units=32, activation=tf.nn.relu)
        self.policy = tf.layers.dense(inputs=self.dense, units=self.a_size, activation=tf.nn.softmax)
        
        self.actions = tf.placeholder(shape=[None, self.a_size], dtype=tf.float32)
        self.discounted_episode_rewards = tf.placeholder(shape=[None,], dtype=tf.float32)
        
        log_prob = tf.log(tf.clip_by_value(self.policy, 0.000001, 0.999999))
        neg_log_responsible_policy = -tf.reduce_sum(tf.multiply(log_prob, self.actions), reduction_indices=1)
        self.loss = tf.reduce_mean(neg_log_responsible_policy * self.discounted_episode_rewards)
        
        trainer = tf.train.AdamOptimizer(learning_rate=learning_rate)
        self.optimize = trainer.minimize(self.loss)

In [27]:
tf.reset_default_graph()

network = PolicyNetwork(s_size, a_size)
init = tf.global_variables_initializer()

In [28]:
sess = tf.Session()
sess.run(init)

In [29]:
num_episodes = 300

for episode in range(num_episodes):
    s = env.reset()
    states = []
    actions = []
    rewards = []
    r_total = 0
    done = False
    
    while not done:
        pi = sess.run(network.policy, feed_dict={network.states: [s]})
        action = np.random.choice(a_size, p=pi[0])
        s1, r, done, _ = env.step(action)
        
        action_vec = np.zeros(a_size)
        action_vec[action] = 1
        
        states.append(s)
        actions.append(action_vec)
        rewards.append(r)
        r_total += r
        
        if done:
            discounted_rewards = discount_rewards(rewards)
            loss, _ = sess.run([network.loss, network.optimize],
                               feed_dict={network.states: states,
                                          network.actions: actions,
                                          network.discounted_episode_rewards: discounted_rewards})
            if episode % 10 == 0:
                print("EPIDOSE {:0>5}: {}".format(episode, r_total))
        
        s = s1

EPIDOSE 00000: 34.0
EPIDOSE 00010: 27.0
EPIDOSE 00020: 43.0
EPIDOSE 00030: 89.0
EPIDOSE 00040: 31.0
EPIDOSE 00050: 14.0
EPIDOSE 00060: 18.0
EPIDOSE 00070: 18.0
EPIDOSE 00080: 31.0
EPIDOSE 00090: 14.0
EPIDOSE 00100: 25.0
EPIDOSE 00110: 20.0
EPIDOSE 00120: 43.0
EPIDOSE 00130: 13.0
EPIDOSE 00140: 36.0
EPIDOSE 00150: 58.0
EPIDOSE 00160: 30.0
EPIDOSE 00170: 50.0
EPIDOSE 00180: 49.0
EPIDOSE 00190: 28.0
EPIDOSE 00200: 22.0
EPIDOSE 00210: 32.0
EPIDOSE 00220: 73.0
EPIDOSE 00230: 93.0
EPIDOSE 00240: 127.0
EPIDOSE 00250: 120.0
EPIDOSE 00260: 175.0
EPIDOSE 00270: 200.0
EPIDOSE 00280: 200.0
EPIDOSE 00290: 200.0


In [30]:
s = env.reset()
# for i in range(3): env.step(0)
r_total = 0
done = False
while True:
    env.render()
    pi = sess.run(network.policy, feed_dict={network.states: [s]})
    a = np.random.choice(a_size, p=pi[0])
    s, r, done, _ = env.step(a)
    r_total += r
    #print(done)
    if done == True:
        print(r_total)
        break

200.0


In [31]:
env.close()